In [ ]:
import gensim.downloader as api
from pprint import pprint
gmodel = api.load('word2vec-google-news-300') # load pre-trained word2vec model

model_info = api.info('word2vec-google-news-300')
model_path = model_info['file_name']
pprint(model_info)
print(f'Model path is = {model_path}')


In [ ]:

result = gmodel.most_similar(positive=['woman', 'king'], negative=['man']) # find the most similar word to woman + king - man
print(result[0]) # print the first result


In [ ]:
result = gmodel.most_similar(positive=['Tokyo', 'france'], negative=['paris']) # find the most similar word to woman + king - man
pprint(result[:5]) # print the first result

In [ ]:
result = gmodel.most_similar(positive=['rome', 'paris'], negative=['italy']) # find the most similar word to woman + king - man
pprint(result[:5]) # print the first result

Now the idea is that taking a `wolf` (feral dog) removing the `dog` part and add a `cat` part we *will end with some sort of feral cat*.

In [ ]:
result = gmodel.most_similar(positive=['cat', 'wolf'], negative=['dog']) # find the most similar word to woman + king - man
pprint(result[:5]) # print the first result

In [ ]:
result = gmodel.most_similar(positive=['weapon', 'wood'], negative=[]) # find the most similar word to woman + king - man
pprint(result[:5]) # print the first result

In [1]:
import torch
from pprint import pprint
from tqdm import tqdm 
from transformers import AutoTokenizer, AutoModel

print(torch.backends.mps.is_available())

# this ensures that the current current PyTorch installation was built with MPS activated.
print(torch.backends.mps.is_built())

device = 'cuda' if torch.cuda.is_available() else 'cpu'

if (device == 'cpu'):
    # ok we are using CPU but we could use apple metal instead
    if torch.backends.mps.is_available():
        print ("Using MPS")
        # device = torch.device('mps')  # use M1 chip if available
    else:
        print ("Using CPU")

# Load model and tokenizer
model_name = "sentence-transformers/distiluse-base-multilingual-cased-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

def cosine_similarity(a, b):
    return torch.nn.functional.cosine_similarity(a, b).item()


c:\develop\github\ai-playground\src\python\langchainVarious\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


False
False


In [2]:
import numpy as np
import requests

# Load word list (assuming you've saved it as a newline-separated text file)

url = 'https://raw.githubusercontent.com/dwyl/english-words/master/words.txt'
response = requests.get(url)

with open('words.txt', 'w') as f:
    f.write(response.text)

with open("words.txt", "r") as f:
    words = f.readlines()
    
words = [word.strip() for word in words]

word_embeddings = []

# Create batches
BATCH_SIZE = 512
num_batches = int(np.ceil(len(words) / BATCH_SIZE))

for i in tqdm(range(num_batches)):
    batch = words[i*BATCH_SIZE: (i+1)*BATCH_SIZE]
    
    # Tokenizing in batch
    inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=32)
    inputs.to(device)
    # Passing through the model
    outputs = model(**inputs).last_hidden_state
    
    # Extract embeddings for each word in the batch
    for j in range(len(batch)):
        word_embedding = outputs[j].mean(dim=0).detach()
        word_embeddings.append(word_embedding)
    
    del inputs, outputs
    torch.cuda.empty_cache()

# Move embeddings back to CPU if needed
word_embeddings = [emb.cpu() for emb in word_embeddings]

pprint (word_embeddings[0])

100%|██████████| 912/912 [01:15<00:00, 12.08it/s]


tensor([ 0.0517, -0.0343,  0.0778,  0.0019,  0.0024, -0.0077,  0.0327,  0.0210,
        -0.0436,  0.0353, -0.0041,  0.1117, -0.0267,  0.0047,  0.0114, -0.0065,
         0.0367,  0.0344,  0.0537, -0.0209,  0.0631,  0.0037,  0.0315, -0.0138,
         0.0600,  0.0095,  0.0313,  0.1338,  0.0926, -0.0573, -0.0919,  0.0053,
        -0.0204,  0.0067,  0.0179,  0.1597, -0.0904,  0.0343,  0.0243,  0.0141,
        -0.0225, -0.0650,  0.0827, -0.0232, -0.0026, -0.0793, -0.0171, -0.0397,
         0.0730,  0.0091,  0.0205, -0.0574, -0.0126,  0.0094, -0.0509, -0.0643,
        -0.0481,  0.0694,  0.0808,  0.0363,  0.0066,  0.0393,  0.0196,  0.0173,
        -0.1359,  0.0173,  0.0638,  0.0540,  0.0068, -0.0518, -0.0224,  0.0717,
         0.0062, -0.0082,  0.0596,  0.0338, -0.0106,  0.0206,  0.0546, -0.0179,
         0.0127, -0.0339,  0.0689, -0.0053,  0.0164, -0.1020,  0.0288, -0.0183,
         0.0498,  0.1885,  0.0189, -0.0025, -0.0225, -0.0240, -0.0098,  0.0206,
         0.0132, -0.0240,  0.1074, -0.02

In [8]:

# Compute analogy vector
# get embeddings
inputs = tokenizer(["woman", "king", "man"], return_tensors="pt", padding=True, truncation=True)
inputs.to(device) # move to GPU if available
outputs = model(**inputs).last_hidden_state

woman = outputs[0].mean(dim=0).detach() # average the embeddings for tokens of "woman"
king = outputs[1].mean(dim=0).detach()  # average the embeddings for tokens of "king"
man = outputs[2].mean(dim=0) .detach()  # average the embeddings for tokens of "man"

del inputs, outputs
torch.cuda.empty_cache()

# compute analogy vector
analogy_vector = woman + king - man
pprint(f"analogy_vector device = {analogy_vector.device}")
analogy_vector = analogy_vector.to("cpu")
pprint(f"analogy_vector device = {analogy_vector.device}")

'analogy_vector device = cuda:0'
'analogy_vector device = cpu'


In [9]:
# Find most similar word
similarities = {}

for word, embedding in zip(words, word_embeddings):
    similarity = cosine_similarity(analogy_vector.unsqueeze(0), embedding.unsqueeze(0))
    similarities[word] = similarity

sorted_similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
print(sorted_similarities[0])  # This should print the most similar word and its similarity score.

cpu
('Queen', 0.6610273718833923)
